In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
# Load the T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Prepare dataset
def preprocess_function(examples):
    inputs = [ex for ex in examples['broken']]
    targets = [ex for ex in examples['corrected']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


In [ ]:

dataset = load_dataset('csv', data_files={'train': 'your_data.csv'}, split='train')
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='./results',
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
)

trainer.train()

# Save the model
model.save_pretrained("fine-tuned-t5")
tokenizer.save_pretrained("fine-tuned-t5")
